# Data Exploration: Toxic Comments

## Set Up: Load modules and training data

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

train = pd.read_csv("train.csv")

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

## Label Exploration

First we need to learn about the class labels. Pandas Categorical class makes it easy to 
cross-tab the different labels on comments, so start by transforming the integer labels to 
categorical. 

In [2]:
for i in range(2,8):
    train.iloc[:,i] = pd.Categorical(train.iloc[:,i])
    
pd.crosstab(train.toxic, train.severe_toxic)

severe_toxic,0,1
toxic,,
0,86614,0
1,8272,965


As we might expect, all severely toxic comments are also toxic comments, but not all toxic 
comments are severe.

In [3]:
pd.crosstab(train.toxic, [train.obscene, train.threat, train.insult, train.identity_hate])

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead tr th {
        text-align: left;
    }

    .dataframe thead tr:last-of-type th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr>
      <th>obscene</th>
      <th colspan="8" halign="left">0</th>
      <th colspan="8" halign="left">1</th>
    </tr>
    <tr>
      <th>threat</th>
      <th colspan="4" halign="left">0</th>
      <th colspan="4" halign="left">1</th>
      <th colspan="4" halign="left">0</th>
      <th colspan="4" halign="left">1</th>
    </tr>
    <tr>
      <th>insult</th>
      <th colspan="2" halign="left">0</th>
      <th colspan="2" halign="left">1</th>
      <th colspan="2" halign="left">0</th>
      <th colspan="2" halign="left">1</th>
      <th colspan="2" halign="left">0</th>
      <th colspan="2" halign="left">1</th>
      <th colspan="2" halign="left">0</th>
      <th colspan="2" halign="left">1</th>
    </tr>
    <tr>
      <th>identity_hate</th>
      <th>0</th>
      <th>1</th>
      <th>0</th>
      <th>1</th>
      <th>0</th>
      <th>1</th>
      <th>0</th>
      <th>1</th>
      <th>0</th>
      <th>1</th>
      <th>0</th>
      <th>1</th>
      <th>0</th>
      <th>1</th>
      <th>0</th>
      <th>1</th>
    </tr>
    <tr>
      <th>toxic</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>86061</td>
      <td>38</td>
      <td>175</td>
      <td>14</td>
      <td>12</td>
      <td>0</td>
      <td>2</td>
      <td>0</td>
      <td>181</td>
      <td>1</td>
      <td>118</td>
      <td>10</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>3449</td>
      <td>74</td>
      <td>754</td>
      <td>70</td>
      <td>79</td>
      <td>3</td>
      <td>8</td>
      <td>3</td>
      <td>1154</td>
      <td>23</td>
      <td>2904</td>
      <td>520</td>
      <td>10</td>
      <td>0</td>
      <td>128</td>
      <td>58</td>
    </tr>
  </tbody>
</table>
</div>

Interestingly, over a third of comments are "civilly" toxic, meaning they are neither obscene, 
insult, threat, nor identity hate, yet they are still disruptive. However, adding these labels 
greatly increases the prevalence of toxicity. The worst cases, the 58 comments that are all 
four of the above, are 100% toxic. 

In [4]:
pd.crosstab(train.severe_toxic, [train.obscene, train.threat, train.insult, train.identity_hate])

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead tr th {
        text-align: left;
    }

    .dataframe thead tr:last-of-type th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr>
      <th>obscene</th>
      <th colspan="8" halign="left">0</th>
      <th colspan="8" halign="left">1</th>
    </tr>
    <tr>
      <th>threat</th>
      <th colspan="4" halign="left">0</th>
      <th colspan="4" halign="left">1</th>
      <th colspan="4" halign="left">0</th>
      <th colspan="4" halign="left">1</th>
    </tr>
    <tr>
      <th>insult</th>
      <th colspan="2" halign="left">0</th>
      <th colspan="2" halign="left">1</th>
      <th colspan="2" halign="left">0</th>
      <th colspan="2" halign="left">1</th>
      <th colspan="2" halign="left">0</th>
      <th colspan="2" halign="left">1</th>
      <th colspan="2" halign="left">0</th>
      <th colspan="2" halign="left">1</th>
    </tr>
    <tr>
      <th>identity_hate</th>
      <th>0</th>
      <th>1</th>
      <th>0</th>
      <th>1</th>
      <th>0</th>
      <th>1</th>
      <th>0</th>
      <th>1</th>
      <th>0</th>
      <th>1</th>
      <th>0</th>
      <th>1</th>
      <th>0</th>
      <th>1</th>
      <th>0</th>
      <th>1</th>
    </tr>
    <tr>
      <th>severe_toxic</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>89488</td>
      <td>109</td>
      <td>921</td>
      <td>81</td>
      <td>83</td>
      <td>3</td>
      <td>10</td>
      <td>3</td>
      <td>1242</td>
      <td>20</td>
      <td>2413</td>
      <td>382</td>
      <td>9</td>
      <td>0</td>
      <td>84</td>
      <td>38</td>
    </tr>
    <tr>
      <th>1</th>
      <td>22</td>
      <td>3</td>
      <td>8</td>
      <td>3</td>
      <td>8</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>93</td>
      <td>4</td>
      <td>609</td>
      <td>148</td>
      <td>2</td>
      <td>0</td>
      <td>45</td>
      <td>20</td>
    </tr>
  </tbody>
</table>
</div>

The cases of "civil" severely toxic comments are much rarer - there are only 22. Generally, a 
smaller portion of comments are severely toxic, no matter what other labels we condition on. 
Even in the worst cases, only 20 of the 58 obscene + threat + insult + identity hate comments 
are severely toxic.

In [5]:
# Convert the categorical data back to integers before correlations
for i in range(2, 8):
    train.iloc[:, i] = pd.to_numeric(train.iloc[:, i])

train.iloc[:, 2:8].corr()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
toxic,1.000000,0.308810,0.677491,0.162967,0.648330,0.259124
severe_toxic,0.308810,1.000000,0.404540,0.133469,0.377450,0.193385
obscene,0.677491,0.404540,1.000000,0.149874,0.744685,0.287794
threat,0.162967,0.133469,0.149874,1.000000,0.157534,0.123971
insult,0.648330,0.377450,0.744685,0.157534,1.000000,0.331922
identity_hate,0.259124,0.193385,0.287794,0.123971,0.331922,1.000000


The correlation matrix is another way of summarizing the relationships between labels,
although here we only see pair-wise correlations rather than the full cross-tabulation.
The story stays the same - all the correlations are positive, and the correlations for 
severe_toxic are always smaller than for toxic. 

The correlation matrix will make for a good sanity check later when making multi-class 
predicitons. We should expect the correlation matrix of the predicted probabilities to look 
very similar to this one, else something is likely awry. 

## What makes a comment toxic?

Let's start out with an overview of the comments' structures.

In [6]:
train[train.comment_text.isnull()]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate


There are no missing values for the comment texts, so let's check for empty strings.

In [7]:
train[train.comment_text == '']

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate


Looks okay. If we find secretly missing values later we can deal with them then.

In [12]:
train['comment_length'] = train.comment_text.str.len()
train.comment_length.describe()

count    95851.000000
mean       395.341864
std        595.102072
min          6.000000
25%         96.000000
50%        206.000000
75%        435.000000
max       5000.000000
Name: comment_length, dtype: float64

The mean is about double the median, so there are some huge comments skewing the data. The 
largest comment is 5000 characters, while the inter-quartile range is only 96 to 435 characters.
Let's see how many comments are 2000 characters or more. 

In [38]:
train = train.sort_values(by="comment_length", ascending=False)
pd.set_option('display.max_colwidth', -1)
train.comment_text.head(1)

94815    JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!!  JIM WALES MUST DIE!!!!!!!!!!!! 

Well, I've only displayed 1 comment, but change this to head(10) or so, and you'll see for
yourself these are very vulgar and spammy. You could probably target these basic spam posts
by targeting a low ratio of unique words to comment length. For the record, you and I both 
surely do not want Jimmy Wales to die!

In [45]:
one_percent = int(np.ceil(train.shape[0] / 100))
train_sub = train.iloc[0:one_percent, :]
train_sub.toxic.value_counts()

0    818
1    141
Name: toxic, dtype: int64

Long comments in general aren't especially toxic. In the above 1% longest comments, still over
80% are not toxic.